In [1]:
!nvidia-smi

Thu Mar 14 16:57:59 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN Xp            Off  | 00000000:01:00.0 Off |                  N/A |
| 23%   28C    P8    16W / 250W |      0MiB / 12195MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [3]:
X_train = np.load("/storage/yw18581/data/train_validation_test/Xy_train_dist.npz")["y"]
y_train = np.load("/storage/yw18581/data/train_validation_test/Xy_train_dist.npz")["dist"]

KeyboardInterrupt: 

In [ ]:
X_val = np.load("/storage/yw18581/data/train_validation_test/Xy_val_dist.npz")["y"]
y_val = np.load("/storage/yw18581/data/train_validation_test/Xy_val_dist.npz")["dist"]

In [ ]:
X_pred = np.load("/storage/yw18581/data/trained_models/UNet_training_generator_1500epochs/Xy_test_predicted_UNet.npz")['y']

In [ ]:
X_pred.shape

In [ ]:
plt.imshow(X_train[66][...,0])
plt.axhline(y=950)
plt.axhline(y=1300)
plt.axvline(x =600)

In [ ]:
plt.imshow(X_pred[3])
plt.axhline(y=950)
plt.axhline(y=1300)
plt.axvline(x =600)

In [ ]:
plt.imshow(X_train[9][...,0][960:1300,600:])


In [ ]:
def cut_X(arr):
    x_cut = arr[:,960:1300,600:]
    x_cut = x_cut[...,0]
    x_cut_out = x_cut.reshape(x_cut.shape[0],x_cut.shape[1]*x_cut.shape[2])
    return x_cut_out

In [ ]:
X_train_cut = cut_X(X_train)

In [ ]:
X_train_cut.shape

In [ ]:
X_val_cut = cut_X(X_val)

In [ ]:
X_val_cut.shape

In [ ]:
def ohe(values):
    values_reshaped = values.reshape(-1, 1)
    onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
    onehot_encoded = onehot_encoder.fit_transform(values_reshaped)
    return onehot_encoded

In [ ]:
y_train_ohe = ohe(y_train)
y_val_ohe = ohe(y_val)

In [ ]:
y_train.shape

In [ ]:
y_train_ohe.shape

In [ ]:
from keras.layers import Input
from keras.layers.core import Dense, Dropout
from keras.models import Model
from keras.losses import categorical_crossentropy, mse
from keras.optimizers import Adadelta, Adam, SGD

In [ ]:
def create_model(input_shape):
    input_layer = Input(shape=input_shape,name='input')
    x = Dense(256, activation='tanh')(input_layer)
    x = Dense(128, activation ='tanh')(x)
    x = Dense(32, activation ='tanh')(x)
    prediction = Dense(4, activation='softmax')(x)
    model = Model(inputs = input_layer, outputs = prediction)
    return model

In [ ]:
model = create_model(input_shape=(y_cut_reshape.shape[1],))

In [ ]:
model.summary()

In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(), metrics=['accuracy'])

In [ ]:
history = model.fit(y_cut_reshape, d_ohe, validation_split=0.2,
                    epochs=100, verbose=True, batch_size=4)

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['loss'],label='training')
plt.plot(history.history['val_loss'],label='validation')
plt.ylim(0,1)
plt.legend(loc='upper right')

In [ ]:
plt.plot(history.history['acc'],label='training')
plt.plot(history.history['val_acc'],label='validation')
plt.ylim(0,1)
plt.legend(loc='upper right')

In [ ]:
def percentage_nonzero(arr):
    return(np.count_nonzero(arr)/np.size(arr)*100)

In [ ]:
percents = np.asarray(list(map(percentage_nonzero, y_cut_reshape)))

In [ ]:
print(np.min(percents), np.max(percents))

In [ ]:
np.where(percents>=14)

In [ ]:
percents[np.where(percents>=14)]

In [ ]:
plt.imshow(y_cut[66])

In [ ]:
def create_model(input_shape):
    input_layer = Input(shape=input_shape,name='input')
    x = Dense(256, activation='sigmoid')(input_layer)
    x = Dense(128, activation ='sigmoid')(x)
    x = Dense(64, activation ='sigmoid')(x)
    prediction = Dense(4, activation='softmax')(x)
    model = Model(inputs = input_layer, outputs = prediction)
    return model

In [ ]:
model = create_model(input_shape=(y_cut_reshape.shape[1],))
model.compile(loss=categorical_crossentropy,
              optimizer=SGD(), metrics=['accuracy'])
model.summary()

In [ ]:
history_sigm = model.fit(y_cut_reshape, d_ohe, validation_split=0.2,
                    epochs=20, verbose=True, batch_size=4)

In [ ]:
def plot_history(history):
    f1 = plt.figure()
    plt.plot(history.history['loss'],label='training')
    plt.plot(history.history['val_loss'],label='validation')
    #plt.ylim(0,1)
    plt.legend(loc='upper right')
    plt.show()
    
    f2 = plt.figure()
    plt.plot(history.history['acc'],label='training')
    plt.plot(history.history['val_acc'],label='validation')
    plt.ylim(0,1)
    plt.legend(loc='upper right')
    plt.show()
    return

In [ ]:
plot_history(history_sigm)

In [ ]:
plot_history(history_sigm)

In [ ]:
def create_model(input_shape):
    input_layer = Input(shape=input_shape,name='input')
    x = Dense(256, activation='sigmoid')(input_layer)
    x = Dense(128, activation ='sigmoid')(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation ='sigmoid')(x)
    prediction = Dense(4, activation='softmax')(x)
    model = Model(inputs = input_layer, outputs = prediction)
    return model


In [ ]:
model = create_model(input_shape=(y_cut_reshape.shape[1],))
model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(), metrics=['accuracy'])
model.summary()


In [ ]:
history_sigm_drop = model.fit(y_cut_reshape, d_ohe, validation_split=0.2,
                    epochs=20, verbose=True, batch_size=4)

In [ ]:
plot_history(history_sigm_drop)

In [ ]:
def create_model(input_shape):
    input_layer = Input(shape=input_shape,name='input')
    x = Dense(256, activation='sigmoid')(input_layer)
    x = Dense(128, activation ='sigmoid')(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation ='sigmoid')(x)
    x = Dropout(0.1)(x)
    x = Dense(32, activation ='sigmoid')(x)
    prediction = Dense(4, activation='softmax')(x)
    model = Model(inputs = input_layer, outputs = prediction)
    return model

model = create_model(input_shape=(y_cut_reshape.shape[1],))
model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(), metrics=['accuracy'])
model.summary()


In [ ]:
history_sigm_drop2 = model.fit(y_cut_reshape, d_ohe, validation_split=0.2,
                    epochs=50, verbose=True, batch_size=4)

In [ ]:
plot_history(history_sigm_drop2)

In [ ]:
def create_model(input_shape):
    input_layer = Input(shape=input_shape,name='input')
    x = Dense(256, activation='sigmoid')(input_layer)
    x = Dense(128, activation ='sigmoid')(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation ='sigmoid')(x)
    prediction = Dense(4, activation='softmax')(x)
    model = Model(inputs = input_layer, outputs = prediction)
    return model

model = create_model(input_shape=(X_train_cut.shape[1],))
model.compile(loss=categorical_crossentropy,
              optimizer=SGD(lr=0.01, momentum=0.9), metrics=['accuracy'])
model.summary()


In [ ]:
history_sigm_drop_train_val = model.fit(X_train_cut, y_train_ohe, validation_data=(X_val_cut, y_val_ohe),
                    epochs=30, verbose=True, batch_size=8, shuffle=True)

In [ ]:
plot_history(history_sigm_drop_train_val)

In [ ]:
def create_model(input_shape):
    input_layer = Input(shape=input_shape,name='input')
    x = Dense(64, activation='sigmoid')(input_layer)
    x = Dropout(0.8)(x)
    prediction = Dense(4, activation='softmax')(x)
    model = Model(inputs = input_layer, outputs = prediction)
    return model

model = create_model(input_shape=(X_train_cut.shape[1],))
model.compile(loss=categorical_crossentropy,
              optimizer=SGD(lr=0.01, momentum=0.9), metrics=['accuracy'])
model.summary()



In [ ]:
history_short = model.fit(X_train_cut, y_train_ohe, validation_data=(X_val_cut, y_val_ohe),
                    epochs=100, verbose=True, batch_size=8, shuffle=True)